In [1]:
# https://charlesdedampierre.github.io/BunkaTopics/index.html
# !pip install bunkatopics
# !pip install xformers
# !pip install triton

In [2]:
import pandas as pd

In [3]:
dataset2 = pd.read_csv("./RBAM-dataset-3-TRAIN.csv", index_col=0)



In [4]:
dataset2Unrelatedfiltered = dataset2.loc[dataset2['relation'] != "neither"]

In [5]:
docs = [b["argTrg"]+" "+b["argSrc"] for a,b in dataset2Unrelatedfiltered.iterrows()] 
# docs = [b["topic"] for a,b in dataset2.iterrows()] 

In [25]:
from bunkatopics import Bunka
from langchain_community.embeddings import HuggingFaceEmbeddings

# very light embedding
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",# We recommend starting with a small model
                                        model_kwargs={"device": "cuda"}, # Or cuda if you have GPU
                                        encode_kwargs={"show_progress_bar": True}, # Show the progress of embeddings
                                        multi_process=True)  # set to True if you have mutliprocessing

# heavier embedding
# embedding_model = HuggingFaceEmbeddings(model_name="infgrad/stella_en_400M_v5",# We recommend starting with a small model
#                                         model_kwargs={"device": "cuda", "trust_remote_code": True}, # Or cuda if you have GPU
#                                         encode_kwargs={"show_progress_bar": True}, # Show the progress of embeddings
#                                         multi_process=True)  # set to True if you have mutliprocessing

# Initialize Bunka with your chosen model and language preference
bunka = Bunka(embedding_model=embedding_model) # You can choose any language you prefer

# Fit Bunka to your text data
bunka.fit(docs)

2024-07-25 16:10:09 - Bunka - INFO - Processing 4803067 tokens
2024-07-25 16:10:10 - Bunka - INFO - Detected language: English
2024-07-25 16:10:10 - Bunka - INFO - Embedding documents... (can take varying amounts of time depending on their size)
2024-07-25 16:10:57 - Bunka - INFO - Reducing the dimensions of embeddings...
2024-07-25 16:12:18 - Bunka - INFO - Extracting meaningful terms from documents...
2024-07-25 16:12:18 - Bunka - INFO - Sampling 1000 documents for term extraction
100%|██████████| 1000/1000 [00:09<00:00, 100.67it/s]


In [26]:
bunka.get_topics(n_clusters=75, name_length=3)# Specify the number of terms to describe each topic

2024-07-25 16:12:30 - Bunka - INFO - Computing the topics


,topic_id,topic_name,size,percent
0,bt-16,capitalism | wealth | bourgeoisie,2656,3.16
1,bt-53,existence | soul | arguments,2372,2.82
2,bt-6,universe | theories | physics,2324,2.77
3,bt-12,tomb | resurrection | disciples,2316,2.76
4,bt-22,vote | parties | turnout,2226,2.65
...,...,...,...,...
67,bt-37,closure | reparations | victims,250,0.30
68,bt-29,sports | cisgender | kg,242,0.29
69,bt-27,pizza | pineapple | meal,174,0.21
70,bt-69,vampire | starvation | defense,72,0.09


In [27]:
viz = bunka.visualize_topics(width=800, height=800, colorscale='YlGnBu', show_text=True)

2024-07-25 16:12:44 - Bunka - INFO - Creating the Bunka Map


In [28]:
viz.write_html("viz.html")

In [29]:
bunka.df_topics_

,topic_id,topic_name,size,percent
0,bt-16,capitalism | wealth | bourgeoisie,2656,3.16
1,bt-53,existence | soul | arguments,2372,2.82
2,bt-6,universe | theories | physics,2324,2.77
3,bt-12,tomb | resurrection | disciples,2316,2.76
4,bt-22,vote | parties | turnout,2226,2.65
...,...,...,...,...
67,bt-37,closure | reparations | victims,250,0.30
68,bt-29,sports | cisgender | kg,242,0.29
69,bt-27,pizza | pineapple | meal,174,0.21
70,bt-69,vampire | starvation | defense,72,0.09


In [30]:
bunka.df_top_docs_per_topic_

,doc_id,content,ranking_per_topic,topic_id,topic_name
0,36010727-07fc-4f0a-9,The professionalisation of sex work may follow...,1,bt-0,workers | assault | responders
1,849cb0a3-18b4-4309-b,"Even if sex work were legal, employers may sti...",2,bt-0,workers | assault | responders
2,b23fa211-4f59-4dc0-9,"In Zurich, where sex work is legal and regulat...",3,bt-0,workers | assault | responders
3,b4f1fb2b-9c17-4fc2-9,Sexual assault against women [327] increased s...,4,bt-0,workers | assault | responders
4,d06dab4d-fac5-4ee3-a,An increase in the disposable income of sex wo...,5,bt-0,workers | assault | responders
...,...,...,...,...,...
814,1002c36d-9930-459b-9,Ecotourism's support of a sustainable economy ...,6,bt-9,car | cars | driving
815,2e5f51a5-fa19-46cb-9,"Most use a car to get to school, which release...",7,bt-9,car | cars | driving
816,1401c526-d847-49ab-9,The context in which the song was written make...,8,bt-9,car | cars | driving
817,1cc852e1-abc9-4ee3-9,There is no meaningful way to compare music fr...,9,bt-9,car | cars | driving
